In [2]:
import numpy as np
import cv2
import matplotlib.pyplot as plt 
from scipy.interpolate import griddata
import pandas as pd

In [11]:
timestamps = pd.read_csv('../Metadata/video_timestamps_7001.csv')
timestamps.head()

,SiteId,BuildingTy,Label,UniqueID,point_crd,join_Type,join_Mode,join_Timestamp(sec),join_Latitude,join_Longitude,join_Altitude,join_Speed(m/s),join_Bearing,join_Accuracy,join_bearing_rad,distance,azimuth
0,6,0,NaN,0.0,Point (538518.7320012 27965.10944211),5,16,180.973,30.250727,-88.139145,-23,7.98,180.9,3,3.157301,28.366146,5.019435
1,6,0,NaN,1.0,Point (538468.10305099 27959.22949709),5,16,183.003,30.250612,-88.139160,-23,5.77,183.8,3,3.207915,21.785987,0.597758
2,6,0,NaN,2.0,Point (538201.88048334 27944.66741691),5,16,1.658,30.250750,-88.142067,33,0.00,0.0,5,0.000000,25.693206,2.497750
3,6,0,NaN,3.0,Point (538160.40282711 28086.37590104),5,16,18.870,30.251871,-88.142143,-14,10.04,0.2,6,0.003491,42.747164,3.921810
4,6,0,NaN,4.0,Point (538457.90057491 28164.76557397),5,16,160.733,30.252123,-88.139511,-27,11.55,94.5,3,1.649336,44.652382,1.570796


In [6]:
timestamps.columns

Index(['SiteId', 'BuildingTy', 'Label', 'UniqueID', 'point_crd', 'join_Type',
       'join_Mode', 'join_Timestamp(sec)', 'join_Latitude', 'join_Longitude',
       'join_Altitude', 'join_Speed(m/s)', 'join_Bearing', 'join_Accuracy',
       'join_bearing_rad', 'distance', 'azimuth'],
      dtype='object')

In [57]:
def camera(picture, outputshape=(400, 400), theta=60, phi=-90, windowsize=(55, 55)):
    
    imgsize=picture.shape[0]
    orix, oriy = picture.shape[0]/2, picture.shape[1]/2
    
    map1 = []
    map2 = []
    
    #print(orix)
    
    # Mapping construction DESTINATION
    for x in range(outputshape[0]):
        for y in range(outputshape[1]):
            map2.append([x,y])
    #print(np.array(map2).shape)
    
    for p in np.linspace(phi-windowsize[1]/2, phi+windowsize[1]/2, outputshape[1]):
        for r in np.linspace(theta-windowsize[0]/2, theta+windowsize[0]/2, outputshape[0]):
            p1 = orix + (r/90*orix) * np.cos(p/180*np.pi)
            p2 = oriy + (r/90*orix) * np.sin(p/180*np.pi)
            map1.append([int(p1), int(p2)])

        
    grid_x, grid_y = np.mgrid[0:outputshape[0]:outputshape[0]*1j, 0:outputshape[1]:outputshape[0]*1j]
    grid_z = griddata(map2, map1, (grid_x, grid_y), method='cubic')
    map_x = np.append([], [ar[:,0] for ar in grid_z]).reshape(outputshape)
    map_y = np.append([], [ar[:,1] for ar in grid_z]).reshape(outputshape)
    map_x_32 = map_x.astype('float32')
    map_y_32 = map_y.astype('float32')
    
    trf = cv2.remap(picture,
                     map_x_32, map_y_32, 
                     cv2.INTER_LINEAR,)
    
    rows,cols = trf.shape[:2]

    M = cv2.getRotationMatrix2D((cols/2,rows/2),-90,1)
    dst = cv2.warpAffine(trf,M,(cols,rows))
    return dst
    

# Left-Right version

In [46]:
# Left- right- front version

def snapshots(videopath, timestamps, uniqueid, bearingcorrect=12):
    vidcap = cv2.VideoCapture(videopath)
    success,image = vidcap.read()

    #frame_count = vidcap.get(cv2.CAP_PROP_FRAME_COUNT)
    #fps=vidcap.get(cv2.CAP_PROP_FPS)
    #duration = frame_count/fps


    for timestamp, uid in zip(timestamps[:3], uniqueid[:3]):
        frame_no = timestamp*1000  #In Milli-seconds
        #print(timestamp, ' - ', frame_no)
        vidcap.set(cv2.CAP_PROP_POS_MSEC,frame_no)
        success, image = vidcap.read()
        
        rows,cols = image.shape[:2]

        M = cv2.getRotationMatrix2D((cols/2,rows/2),bearingcorrect,1)
        image = cv2.warpAffine(image,M,(cols,rows))
        
        cv2.imwrite("../exports/{} - {} - frame.jpg".format(int(uid),int(timestamp*1000)), image)
        
        left = camera(image, theta=55, phi=0)
        cv2.imwrite("../exports/{} - {} - left.jpg".format(int(uid),int(timestamp*1000)), left)
        
        right = camera(image, theta=55, phi=180)
        cv2.imwrite("../exports/{} - {} - right.jpg".format(int(uid), int(timestamp*1000)), right)
        
        front = camera(image, theta=55, phi=-90)
        cv2.imwrite("../exports/{} - {} - front.jpg".format(int(uid), int(timestamp*1000)), front)
        
        print('uid:{} Done'.format(int(uid)))
        
    
    print('done')

In [47]:
snapshots('../video/FLY70001.MP4', timestamps['join_Timestamp(sec)'], timestamps['UniqueID'], bearingcorrect=12)

uid:0 Done
uid:1 Done
uid:2 Done
uid:3 Done
uid:4 Done
uid:5 Done
uid:6 Done
uid:7 Done
uid:8 Done
uid:9 Done
uid:10 Done
uid:11 Done
uid:12 Done
uid:13 Done
uid:14 Done
uid:15 Done
uid:16 Done
uid:17 Done
uid:18 Done
uid:19 Done
done


# Angle version

In [16]:
df = timestamps[['UniqueID', 'join_Timestamp(sec)', 'azimuth', 'join_bearing_rad']]
df.columns = ['UniqueID', 'timestamp', 'azshot', 'azcamera']
df.head()

,UniqueID,timestamp,azshot,azcamera
0,0.0,180.973,5.019435,3.157301
1,1.0,183.003,0.597758,3.207915
2,2.0,1.658,2.497750,0.000000
3,3.0,18.870,3.921810,0.003491
4,4.0,160.733,1.570796,1.649336


In [33]:
for index, house in df[:3].iterrows():
    print(int(house['UniqueID']))

0
1
2


In [59]:
# Angle version

def snapshots(videopath, df):
    vidcap = cv2.VideoCapture(videopath)
    success, image = vidcap.read()


    for index, house in df.iloc[[29,65,67, 68]].iterrows():
        frame_no = house['timestamp']*1000  #In Milli-seconds
        uid = int(house['UniqueID'])
        angle = (house['azshot'] - house['azcamera'])*90/np.pi - 90
        
        vidcap.set(cv2.CAP_PROP_POS_MSEC,frame_no)
        success, image = vidcap.read()
        
        cv2.imwrite("../exports/{} - {} - frame.jpg".format(uid, int(frame_no)), image)
        
        
        house = camera(image, theta=55, phi=angle)
        cv2.imwrite("../exports/{} - {} - house.jpg".format(uid, int(frame_no)), house)

        
        #front = camera(image, theta=55, phi=-90)
        #cv2.imwrite("../exports/{} - {} - front.jpg".format(int(uid), int(timestamp*1000)), front)
        
        print('uid:{} Done'.format(uid))
        
    
    print('done')
    
snapshots('../video/FLY70001.MP4', df)

uid:29 Done
uid:65 Done
uid:67 Done
uid:68 Done
done


In [52]:
df[df['UniqueID']==29]

,UniqueID,timestamp,azshot,azcamera
29,29.0,249.945,5.944354,1.555088


In [55]:
df.iloc[[29,35,68]]

,UniqueID,timestamp,azshot,azcamera
29,29.0,249.945,5.944354,1.555088
35,35.0,208.400,1.918414,0.022689
68,68.0,284.355,5.727829,1.577778


# House panoramique

In [61]:
timestamps = pd.read_csv('../Metadata/video_house_pan_7001.csv')
timestamps.head()

,Type,Mode,Timestamp(sec),Latitude,Longitude,Altitude,Speed(m/s),Bearing,Accuracy,bearing_rad,join_SiteId,join_BuildingTy,join_Label,join_UniqueID,join_point_crd,distance,shot_bearing,bearing_shot
0,5,16,0.651,30.25075,-88.14206,40,0.00,0.0,5,0.00000,6,0,NaN,12,Point (538231.9606252 27958.57451765),24.442544,2.015583,2
1,5,16,1.658,30.25075,-88.14207,33,0.00,0.0,5,0.00000,6,0,NaN,12,Point (538231.9606252 27958.57451765),25.197320,2.010788,2
2,5,16,2.674,30.25075,-88.14207,30,0.00,0.0,5,0.00000,6,0,NaN,2,Point (538201.88048334 27944.66741691),25.703521,3.399251,3
3,5,16,3.683,30.25075,-88.14206,31,0.55,43.2,5,0.75398,6,0,NaN,12,Point (538231.9606252 27958.57451765),24.718899,2.038811,2
4,5,16,4.690,30.25076,-88.14205,32,1.64,28.7,5,0.50091,6,0,NaN,12,Point (538231.9606252 27958.57451765),24.671048,2.098144,2


In [63]:
df = timestamps[['join_UniqueID', 'Timestamp(sec)', 'shot_bearing', 'bearing_rad']]
df.columns = ['UniqueID', 'timestamp', 'azshot', 'azcamera']
df.head()

,UniqueID,timestamp,azshot,azcamera
0,12,0.651,2.015583,0.00000
1,12,1.658,2.010788,0.00000
2,2,2.674,3.399251,0.00000
3,12,3.683,2.038811,0.75398
4,12,4.690,2.098144,0.50091


In [68]:
# Angle version

def snapshots(videopath, df, houseid):
    vidcap = cv2.VideoCapture(videopath)
    success, image = vidcap.read()

    subset = df[df['UniqueID'] == houseid]

    for index, house in subset.iterrows():
        frame_no = house['timestamp']*1000  #In Milli-seconds
        uid = int(house['UniqueID'])
        angle = (house['azshot'] - house['azcamera'])*90/np.pi - 90
        
        vidcap.set(cv2.CAP_PROP_POS_MSEC,frame_no)
        success, image = vidcap.read()
        
        cv2.imwrite("../exports/zz - {} - {} - frame.jpg".format(uid, int(frame_no)), image)
        
        
        house = camera(image, theta=55, phi=angle)
        cv2.imwrite("../exports/{} - {} - house.jpg".format(uid, int(frame_no)), house)

        
        #front = camera(image, theta=55, phi=-90)
        #cv2.imwrite("../exports/{} - {} - front.jpg".format(int(uid), int(timestamp*1000)), front)
        
        print('uid:{} - tstp:{} Done'.format(uid, frame_no))
        
    
    print('done')
    
snapshots('../video/FLY70001.MP4', df, 77)

uid:77 - tstp:27986.0 Done
uid:77 - tstp:29001.0 Done
uid:77 - tstp:30010.0 Done
done


In [69]:
df[df['UniqueID'] == 77]

,UniqueID,timestamp,azshot,azcamera
27,77,27.986,0.288203,1.57603
28,77,29.001,6.265179,1.57952
29,77,30.010,5.938141,1.56905
